In [1]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pickle
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configuration
DATA_DIR = Path("analysis_data")
CHECKPOINT_DIR = DATA_DIR / "checkpoints"
DATA_DIR.mkdir(exist_ok=True)
CHECKPOINT_DIR.mkdir(exist_ok=True)

In [2]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pickle
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configuration
DATA_DIR = Path("analysis_data")
CHECKPOINT_DIR = DATA_DIR / "checkpoints"
DATA_DIR.mkdir(exist_ok=True)
CHECKPOINT_DIR.mkdir(exist_ok=True)

class CheckpointManager:
    def __init__(self, process_name, parameters=None):
        self.process_name = process_name
        self.parameters = parameters or {}
        # Create unique checkpoint file based on parameters
        param_hash = hash(str(sorted(self.parameters.items())))
        self.checkpoint_file = CHECKPOINT_DIR / f"{process_name}_{param_hash}_progress.json"
        self.progress = self.load_progress()
    
    def load_progress(self):
        if self.checkpoint_file.exists():
            with open(self.checkpoint_file, 'r') as f:
                data = json.load(f)
                # Verify parameters match
                if data.get("parameters") == self.parameters:
                    return data
                else:
                    print(f"Parameters changed for {self.process_name}, starting fresh")
                    return self._create_new_progress()
        return self._create_new_progress()
    
    def _create_new_progress(self):
        return {
            "parameters": self.parameters,
            "completed_chunks": [], 
            "status": "not_started", 
            "last_chunk": -1
        }
    
    def save_progress(self, chunk_info):
        self.progress["completed_chunks"].append(chunk_info)
        self.progress["last_chunk"] = len(self.progress["completed_chunks"]) - 1
        self.progress["status"] = "in_progress"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def mark_completed(self):
        self.progress["status"] = "completed"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def is_chunk_completed(self, chunk_id):
        return any(chunk["chunk_id"] == chunk_id for chunk in self.progress["completed_chunks"])
    
    def get_completed_chunks(self):
        return [chunk["filename"] for chunk in self.progress["completed_chunks"]]

def save_dataframe(df, filename_base):
    """Save dataframe with best available format"""
    try:
        # Try parquet first
        parquet_file = DATA_DIR / f"{filename_base}.parquet"
        df.to_parquet(parquet_file, index=False)
        return parquet_file
    except ImportError:
        # Fallback to pickle
        pickle_file = DATA_DIR / f"{filename_base}.pkl"
        with open(pickle_file, 'wb') as f:
            pickle.dump(df, f)
        return pickle_file

def load_dataframe(filename_base):
    """Load dataframe from best available format"""
    for ext, loader in [('.parquet', pd.read_parquet), ('.pkl', lambda x: pickle.load(open(x, 'rb')))]:
        file_path = DATA_DIR / f"{filename_base}{ext}"
        if file_path.exists():
            return loader(file_path)
    return None

def connect_server(ip, database):
    conn_str = (
        "Driver={SQL Server};"
        "Server=" + ip + ";" + 
        "Database=" + database + ";" + 
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    return conn, cursor

def sql_to_pandas(conn, cursor, sql_string):
    cursor.execute(sql_string)
    rows = cursor.fetchall()
    if rows:
        return pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])
    return pd.DataFrame()

def close_conn(conn, cursor):
    cursor.close()
    conn.close()


In [3]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pickle
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configuration
DATA_DIR = Path("analysis_data")
CHECKPOINT_DIR = DATA_DIR / "checkpoints"
DATA_DIR.mkdir(exist_ok=True)
CHECKPOINT_DIR.mkdir(exist_ok=True)

class CheckpointManager:
    def __init__(self, process_name, parameters=None):
        self.process_name = process_name
        self.parameters = parameters or {}
        param_hash = hash(str(sorted(self.parameters.items())))
        self.checkpoint_file = CHECKPOINT_DIR / f"{process_name}_{param_hash}_progress.json"
        self.progress = self.load_progress()
    
    def load_progress(self):
        if self.checkpoint_file.exists():
            with open(self.checkpoint_file, 'r') as f:
                data = json.load(f)
                if data.get("parameters") == self.parameters:
                    return data
                else:
                    print(f"Parameters changed for {self.process_name}, starting fresh")
                    return self._create_new_progress()
        return self._create_new_progress()
    
    def _create_new_progress(self):
        return {
            "parameters": self.parameters,
            "completed_chunks": [], 
            "status": "not_started", 
            "last_chunk": -1
        }
    
    def save_progress(self, chunk_info):
        self.progress["completed_chunks"].append(chunk_info)
        self.progress["last_chunk"] = len(self.progress["completed_chunks"]) - 1
        self.progress["status"] = "in_progress"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def mark_completed(self):
        self.progress["status"] = "completed"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def is_chunk_completed(self, chunk_id):
        return any(chunk["chunk_id"] == chunk_id for chunk in self.progress["completed_chunks"])
    
    def get_completed_chunks(self):
        return [chunk["filename"] for chunk in self.progress["completed_chunks"]]

def save_dataframe(df, filename_base):
    """Save dataframe with best available format"""
    try:
        parquet_file = DATA_DIR / f"{filename_base}.parquet"
        df.to_parquet(parquet_file, index=False)
        return parquet_file
    except ImportError:
        pickle_file = DATA_DIR / f"{filename_base}.pkl"
        with open(pickle_file, 'wb') as f:
            pickle.dump(df, f)
        return pickle_file

def load_dataframe(filename_base):
    """Load dataframe from best available format"""
    for ext, loader in [('.parquet', pd.read_parquet), ('.pkl', lambda x: pickle.load(open(x, 'rb')))]:
        file_path = DATA_DIR / f"{filename_base}{ext}"
        if file_path.exists():
            return loader(file_path)
    return None

def connect_server(ip, database):
    conn_str = (
        "Driver={SQL Server};"
        "Server=" + ip + ";" + 
        "Database=" + database + ";" + 
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    return conn, cursor

def sql_to_pandas(conn, cursor, sql_string):
    cursor.execute(sql_string)
    rows = cursor.fetchall()
    if rows:
        return pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])
    return pd.DataFrame()

def close_conn(conn, cursor):
    cursor.close()
    conn.close()

def analyze_loyalty_bank_cards_corrected(facts_df, bank_df):
    """
    CORRECTED analysis using client_id as bank card identifier
    
    Key changes:
    - reference_number is ONLY used to link tables
    - client_id is the actual bank card number
    - All grouping and filtering is done by customer_id + client_id
    
    Logic:
    1. Aggregate facts by cheque_id to get cheque totals
    2. Join with bank_df using reference_number to get client_id
    3. Validate each customer_id + client_id combination (>=3 cheques OR >$35 total)
    4. Keep customers with <=2 different client_ids from <=2 different banks
    5. If customer has 2 client_ids, they MUST be from 2 different banks
    6. Remove shared client_ids (used by multiple customers)
    """
    print("\n=== CORRECTED LOYALTY BANK CARD ANALYSIS (using client_id) ===")
    
    if facts_df is None or bank_df is None or facts_df.empty or bank_df.empty:
        print("Invalid input data")
        return None, None
    
    # ========================================================================
    # STEP 0: AGGREGATE FACTS DATA BY CHEQUE_ID
    # ========================================================================
    print("\n--- STEP 0: AGGREGATING FACTS DATA BY CHEQUE ---")
    
    print(f"Raw transaction lines: {len(facts_df):,}")
    print(f"Unique cheques: {facts_df['cheque_id'].nunique():,}")
    print(f"Unique customers: {facts_df['customer_id'].nunique():,}")
    print(f"Unique reference numbers: {facts_df['reference_number'].nunique():,}")
    
    cheque_totals = facts_df.groupby(['cheque_id', 'customer_id', 'reference_number']).agg({
        'total_price': 'sum'
    }).reset_index()
    
    cheque_totals.columns = ['cheque_id', 'customer_id', 'reference_number', 'cheque_total']
    
    print(f"Cheque-level data: {len(cheque_totals):,} records")
    print(f"Average spending per cheque: ${cheque_totals['cheque_total'].mean():.2f}")
    
    # ========================================================================
    # STEP 1: JOIN WITH BANK DATA TO GET CLIENT_ID (BANK CARD NUMBER)
    # ========================================================================
    print("\n--- STEP 1: JOINING WITH BANK DATA TO GET CLIENT_ID ---")
    
    cheque_totals['reference_number'] = cheque_totals['reference_number'].astype(str)
    bank_df['reference_number'] = bank_df['reference_number'].astype(str)
    
    # Merge using reference_number to get client_id
    merged_df = cheque_totals.merge(
        bank_df[['reference_number', 'client_id', 'bank_name', 'client']], 
        on='reference_number', 
        how='inner'
    )
    
    print(f"After join:")
    print(f"  Successfully merged cheque records: {len(merged_df):,}")
    print(f"  Unique customers with bank data: {merged_df['customer_id'].nunique():,}")
    print(f"  Unique bank cards (client_id): {merged_df['client_id'].nunique():,}")
    print(f"  Unique reference numbers used for linking: {merged_df['reference_number'].nunique():,}")
    print(f"  Available banks: {sorted(merged_df['bank_name'].unique())}")
    
    if len(merged_df) == 0:
        print("No matching data found!")
        return None, None
    
    # ========================================================================
    # STEP 2: VALIDATE CUSTOMER-CARD COMBINATIONS (using client_id)
    # ========================================================================
    print("\n--- STEP 2: VALIDATING CUSTOMER-CARD COMBINATIONS ---")
    print("Validation: customer_id must have >=3 cheques OR >$35 total spending per client_id")
    
    # Group by customer_id + client_id to get usage stats
    customer_card_usage = merged_df.groupby(['customer_id', 'client_id']).agg({
        'cheque_id': 'nunique',
        'cheque_total': 'sum',
        'bank_name': 'first',
        'client': 'first',
        'reference_number': lambda x: list(x.unique())  # Track which reference numbers link to this client_id
    }).reset_index()
    
    customer_card_usage.columns = ['customer_id', 'client_id', 'cheque_count', 
                                    'total_spending', 'bank_name', 'client', 'reference_numbers']
    
    print(f"Customer-card combinations: {len(customer_card_usage):,}")
    print(f"Average cheques per combination: {customer_card_usage['cheque_count'].mean():.1f}")
    print(f"Average spending per combination: ${customer_card_usage['total_spending'].mean():.2f}")
    
    # Apply validation: >=3 cheques OR >$35 spending
    valid_customer_cards = customer_card_usage[
        (customer_card_usage['cheque_count'] >= 3) | 
        (customer_card_usage['total_spending'] > 35)
    ].copy()
    
    invalid_customer_cards = customer_card_usage[
        (customer_card_usage['cheque_count'] < 3) & 
        (customer_card_usage['total_spending'] <= 35)
    ].copy()
    
    print(f"Valid customer-card combinations: {len(valid_customer_cards):,}")
    print(f"Invalid customer-card combinations: {len(invalid_customer_cards):,}")
    print(f"Validation rate: {len(valid_customer_cards)/len(customer_card_usage)*100:.1f}%")
    
    # ========================================================================
    # STEP 3: FILTER TO BOG AND TBC BANKS ONLY
    # ========================================================================
    print("\n--- STEP 3: FILTERING TO BOG AND TBC BANKS ---")
    
    target_banks = ['BOG', 'TBC']
    bog_tbc_valid_cards = valid_customer_cards[
        valid_customer_cards['bank_name'].isin(target_banks)
    ].copy()
    
    print(f"Valid combinations with BOG/TBC banks: {len(bog_tbc_valid_cards):,}")
    print("Bank distribution:")
    bank_dist = bog_tbc_valid_cards['bank_name'].value_counts()
    for bank, count in bank_dist.items():
        print(f"  {bank}: {count:,} combinations")
    
    if len(bog_tbc_valid_cards) == 0:
        print("No valid BOG/TBC combinations found!")
        return None, None
    
    # ========================================================================
    # STEP 4: FILTER CUSTOMERS WITH <=2 VALID CARDS FROM <=2 BANKS
    # ========================================================================
    print("\n--- STEP 4: FILTERING CUSTOMERS BY CARD AND BANK LIMITS ---")
    print("Requirements:")
    print("  - <=2 different valid client_ids from <=2 different banks")
    print("  - If 2 client_ids, they MUST be from 2 different banks")
    
    customer_summary = bog_tbc_valid_cards.groupby('customer_id').agg({
        'client_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': ['nunique', lambda x: list(x.unique())],
        'cheque_count': 'sum',
        'total_spending': 'sum'
    }).reset_index()
    
    customer_summary.columns = ['customer_id', 'valid_cards_count', 'valid_cards_list',
                                'different_banks_count', 'different_banks_list', 
                                'total_cheques', 'total_spending']
    
    print(f"Customers with valid BOG/TBC cards: {len(customer_summary):,}")
    
    # Apply filtering with constraints
    eligible_customers = customer_summary[
        (customer_summary['valid_cards_count'] <= 2) & 
        (customer_summary['different_banks_count'] <= 2) &
        (
            (customer_summary['valid_cards_count'] == 1) |
            ((customer_summary['valid_cards_count'] == 2) & (customer_summary['different_banks_count'] == 2))
        )
    ].copy()
    
    print(f"After filtering: {len(eligible_customers):,} customers meeting requirements")
    
    if len(eligible_customers) == 0:
        print("No customers meet the criteria!")
        return None, None
    
    print("\nValid cards per customer distribution:")
    cards_after = eligible_customers['valid_cards_count'].value_counts().sort_index()
    for cards, count in cards_after.items():
        print(f"  {count:,} customers have {cards} valid card(s)")
    
    # ========================================================================
    # STEP 5: REMOVE SHARED CLIENT_IDS
    # ========================================================================
    print("\n--- STEP 5: REMOVING SHARED CLIENT_IDS ---")
    print("Remove client_ids used by multiple customers, but keep customers with remaining cards")
    
    eligible_customer_ids = set(eligible_customers['customer_id'])
    eligible_customer_cards = bog_tbc_valid_cards[
        bog_tbc_valid_cards['customer_id'].isin(eligible_customer_ids)
    ].copy()
    
    # Check how many customers use each client_id
    card_usage_analysis = eligible_customer_cards.groupby('client_id').agg({
        'customer_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': 'first',
        'cheque_count': 'sum',
        'total_spending': 'sum'
    }).reset_index()
    
    card_usage_analysis.columns = ['client_id', 'customer_count', 'customer_list',
                                   'bank_name', 'total_cheques', 'total_spending']
    
    shared_cards = card_usage_analysis[
        card_usage_analysis['customer_count'] > 1
    ]['client_id'].tolist()
    
    unique_cards = card_usage_analysis[
        card_usage_analysis['customer_count'] == 1
    ]['client_id'].tolist()
    
    print(f"Cards (client_ids) used by single customer: {len(unique_cards):,}")
    print(f"Cards (client_ids) shared by multiple customers: {len(shared_cards):,}")
    
    # Remove shared cards
    clean_customer_cards = eligible_customer_cards[
        ~eligible_customer_cards['client_id'].isin(shared_cards)
    ].copy()
    
    print(f"Customer-card combinations after removing shared cards: {len(clean_customer_cards):,}")
    
    # ========================================================================
    # STEP 6: CREATE FINAL CLEAN DATASET
    # ========================================================================
    print("\n--- STEP 6: CREATING FINAL CLEAN DATASET ---")
    
    final_customer_summary = clean_customer_cards.groupby('customer_id').agg({
        'client_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': ['nunique', lambda x: list(x.unique())],
        'cheque_count': 'sum',
        'total_spending': 'sum',
        'client': lambda x: list(x.unique()),
        'reference_numbers': lambda x: list(set([ref for refs in x for ref in refs]))
    }).reset_index()
    
    final_customer_summary.columns = ['customer_id', 'final_cards_count', 'final_cards_list',
                                      'different_banks_count', 'different_banks_list', 
                                      'total_cheques', 'total_spending', 'clients_list', 'reference_numbers']
    
    final_customer_summary = final_customer_summary[
        final_customer_summary['final_cards_count'] > 0
    ].copy()
    
    print(f"Final clean customers: {len(final_customer_summary):,}")
    
    # Verify constraints
    constraint_violations = final_customer_summary[
        (final_customer_summary['final_cards_count'] == 2) & 
        (final_customer_summary['different_banks_count'] != 2)
    ]
    
    if len(constraint_violations) > 0:
        print(f"WARNING: {len(constraint_violations)} customers have 2 cards from same bank")
        final_customer_summary = final_customer_summary[
            ~final_customer_summary['customer_id'].isin(constraint_violations['customer_id'])
        ].copy()
        print(f"Removed violations, final customers: {len(final_customer_summary):,}")
    
    print("\nFINAL DISTRIBUTION - Cards per customer:")
    final_cards_dist = final_customer_summary['final_cards_count'].value_counts().sort_index()
    for cards, count in final_cards_dist.items():
        print(f"  {count:,} customers have {cards} card(s)")
    
    # Final verification
    max_cards = final_customer_summary['final_cards_count'].max()
    max_banks = final_customer_summary['different_banks_count'].max()
    two_cards_same_bank = len(final_customer_summary[
        (final_customer_summary['final_cards_count'] == 2) & 
        (final_customer_summary['different_banks_count'] == 1)
    ])
    
    # Check for shared cards in final
    final_customer_ids = set(final_customer_summary['customer_id'])
    final_clean_cards = clean_customer_cards[
        clean_customer_cards['customer_id'].isin(final_customer_ids)
    ].copy()
    
    final_card_sharing = final_clean_cards.groupby('client_id')['customer_id'].nunique()
    shared_in_final = (final_card_sharing > 1).sum()
    
    print(f"\nFINAL VERIFICATION:")
    print(f"  Maximum cards per customer: {max_cards} (requirement: <=2) {'PASS' if max_cards <= 2 else 'FAIL'}")
    print(f"  Maximum banks per customer: {max_banks} (requirement: <=2) {'PASS' if max_banks <= 2 else 'FAIL'}")
    print(f"  Customers with 2 cards from same bank: {two_cards_same_bank} (requirement: 0) {'PASS' if two_cards_same_bank == 0 else 'FAIL'}")
    print(f"  Shared cards in final dataset: {shared_in_final} (requirement: 0) {'PASS' if shared_in_final == 0 else 'FAIL'}")
    
    all_passed = max_cards <= 2 and max_banks <= 2 and two_cards_same_bank == 0 and shared_in_final == 0
    print(f"  OVERALL: {'ALL REQUIREMENTS SATISFIED' if all_passed else 'REQUIREMENTS NOT MET'}")
    
    # ========================================================================
    # CREATE CUSTOMER-CARD PAIRS OUTPUT
    # ========================================================================
    print("\n--- CREATING CUSTOMER-CARD PAIRS ---")
    
    # Create simple pairs dataframe
    customer_card_pairs = final_clean_cards[['customer_id', 'client_id', 'bank_name']].drop_duplicates()
    print(f"Total customer-card pairs: {len(customer_card_pairs):,}")
    
    # Verify each customer has 1 or 2 cards
    pairs_per_customer = customer_card_pairs.groupby('customer_id').size()
    print(f"Pairs per customer: min={pairs_per_customer.min()}, max={pairs_per_customer.max()}")
    
    results = {
        'raw_transaction_lines': len(facts_df),
        'total_cheques': len(cheque_totals),
        'total_loyalty_customers': cheque_totals['customer_id'].nunique(),
        'customers_with_bank_data': merged_df['customer_id'].nunique(),
        'total_customer_card_combinations': len(customer_card_usage),
        'valid_customer_card_combinations': len(valid_customer_cards),
        'bog_tbc_valid_combinations': len(bog_tbc_valid_cards),
        'eligible_customers_before_shared_removal': len(eligible_customers),
        'shared_cards_count': len(shared_cards),
        'final_clean_customers': len(final_customer_summary),
        'final_clean_combinations': len(final_clean_cards),
        
        # DataFrames
        'cheque_totals': cheque_totals,
        'customer_card_usage': customer_card_usage,
        'valid_customer_cards': valid_customer_cards,
        'invalid_customer_cards': invalid_customer_cards,
        'bog_tbc_valid_cards': bog_tbc_valid_cards,
        'eligible_customers': eligible_customers,
        'card_usage_analysis': card_usage_analysis,
        'shared_cards_info': card_usage_analysis[card_usage_analysis['customer_count'] > 1],
        'shared_cards_list': shared_cards,
        'final_clean_cards': final_clean_cards,
        'final_customer_summary': final_customer_summary,
        'customer_card_pairs': customer_card_pairs,  # NEW: Simple pairs output
        'merged_data': merged_df,
        
        # Distributions
        'final_cards_distribution': final_cards_dist
    }
    
    return results, final_customer_summary

def save_corrected_results(results, output_dir="./data"):
    """Save all results to CSV files with focus on customer-card pairs"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    saved_files = []
    
    try:
        # PRIMARY OUTPUT: Customer-Card Pairs
        if 'customer_card_pairs' in results and not results['customer_card_pairs'].empty:
            file_path = output_path / "customer_card_pairs.csv"
            results['customer_card_pairs'].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
            print(f"\nPRIMARY OUTPUT SAVED: {file_path}")
            print(f"  Rows: {len(results['customer_card_pairs']):,}")
            print(f"  Columns: {list(results['customer_card_pairs'].columns)}")
        
        # SECONDARY OUTPUT: Customer summary with all details
        if 'final_customer_summary' in results and not results['final_customer_summary'].empty:
            summary_csv = results['final_customer_summary'].copy()
            
            # Convert list columns to strings
            summary_csv['final_cards_list_str'] = summary_csv['final_cards_list'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            summary_csv['different_banks_list_str'] = summary_csv['different_banks_list'].apply(
                lambda x: ', '.join(x) if isinstance(x, list) else str(x))
            summary_csv['clients_list_str'] = summary_csv['clients_list'].apply(
                lambda x: ', '.join(x) if isinstance(x, list) else str(x))
            summary_csv['reference_numbers_str'] = summary_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x[:20])) + (f' (+{len(x)-20} more)' if len(x) > 20 else '') if isinstance(x, list) else str(x))
            
            csv_columns = ['customer_id', 'final_cards_count', 'different_banks_count', 'total_cheques', 
                          'total_spending', 'final_cards_list_str', 'different_banks_list_str', 
                          'clients_list_str', 'reference_numbers_str']
            
            file_path = output_path / "final_clean_customers_summary.csv"
            summary_csv[csv_columns].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # DETAILED OUTPUT: All customer-card combinations with full details
        if 'final_clean_cards' in results and not results['final_clean_cards'].empty:
            clean_cards_csv = results['final_clean_cards'].copy()
            clean_cards_csv['reference_numbers_str'] = clean_cards_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            output_cols = ['customer_id', 'client_id', 'bank_name', 'cheque_count', 
                          'total_spending', 'client', 'reference_numbers_str']
            
            file_path = output_path / "customer_card_combinations_detailed.csv"
            clean_cards_csv[output_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Shared cards that were removed
        if 'shared_cards_info' in results and not results['shared_cards_info'].empty:
            file_path = output_path / "shared_cards_removed.csv"
            shared_csv = results['shared_cards_info'].copy()
            shared_csv['customer_list_str'] = shared_csv['customer_list'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            csv_cols = ['client_id', 'customer_count', 'bank_name', 'total_cheques', 
                       'total_spending', 'customer_list_str']
            shared_csv[csv_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Invalid customer-card combinations
        if 'invalid_customer_cards' in results and not results['invalid_customer_cards'].empty:
            file_path = output_path / "invalid_customer_card_combinations.csv"
            invalid_csv = results['invalid_customer_cards'].copy()
            invalid_csv['reference_numbers_str'] = invalid_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            output_cols = ['customer_id', 'client_id', 'cheque_count', 'total_spending', 
                          'bank_name', 'reference_numbers_str']
            invalid_csv[output_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Analysis summary
        analysis_summary = {
            'total_loyalty_customers': results['total_loyalty_customers'],
            'final_clean_customers': results['final_clean_customers'],
            'success_rate_percent': results['final_clean_customers']/results['total_loyalty_customers']*100,
            'shared_cards_removed': results['shared_cards_count'],
            'validation_rate_percent': results['valid_customer_card_combinations']/results['total_customer_card_combinations']*100
        }
        
        file_path = output_path / "analysis_summary.csv"
        pd.DataFrame([analysis_summary]).to_csv(file_path, index=False)
        saved_files.append(str(file_path))
        
        print(f"\nSAVED {len(saved_files)} FILES:")
        for file_path in saved_files:
            print(f"   {file_path}")
                
    except Exception as e:
        print(f"Error saving files: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    return True

def run_full_analysis():
    """Run complete analysis"""
    print("RUNNING COMPLETE ANALYSIS - WITH CLIENT_ID")
    print("="*60)
    
    facts_df = load_dataframe("facts_data_2025_combined_customer_id")
    bank_df = load_dataframe("bank_data_combined")
    
    if facts_df is None or bank_df is None:
        print("No saved data found, loading from database...")
        # Add your data loading logic here
        return None
    else:
        print("Using existing saved data")
        print(f"Facts data: {len(facts_df):,} records")
        print(f"Bank data: {len(bank_df):,} records")
    
    if facts_df is None or bank_df is None:
        print("Could not load data")
        return None
    
    # Run analysis
    results, summary = analyze_loyalty_bank_cards_corrected(facts_df, bank_df)
    
    if results is None:
        print("Analysis failed")
        return None
    
    # Save results
    save_corrected_results(results)
    
    return results, summary

def show_usage():
    print("""
USAGE - CORRECTED VERSION WITH CLIENT_ID AS BANK CARD NUMBER

KEY CORRECTION:
- reference_number is ONLY used to link facts and bank tables
- client_id is the actual bank card number
- All analysis and outputs are based on customer_id + client_id pairs

RUN ANALYSIS:
>>> results, summary = run_full_analysis()

OUTPUT FILES:
1. customer_card_pairs.csv - PRIMARY OUTPUT
   - Simple table: customer_id, client_id, bank_name
   - Each customer has 1 or 2 rows (cards)
   - If 2 cards, they are from different banks

2. final_clean_customers_summary.csv
   - One row per customer with aggregated information
   
3. customer_card_combinations_detailed.csv
   - Full details including spending, cheque count, etc.

CONSTRAINTS ENFORCED:
- Each customer has <=2 client_ids (bank cards) from <=2 banks
- If 2 cards, they must be from 2 different banks (BOG or TBC)
- Only cards with >=3 transactions OR >$35 spending
- Shared cards (used by multiple customers) are removed
""")

if __name__ == "__main__":
    show_usage()


USAGE - CORRECTED VERSION WITH CLIENT_ID AS BANK CARD NUMBER

KEY CORRECTION:
- reference_number is ONLY used to link facts and bank tables
- client_id is the actual bank card number
- All analysis and outputs are based on customer_id + client_id pairs

RUN ANALYSIS:
>>> results, summary = run_full_analysis()

OUTPUT FILES:
1. customer_card_pairs.csv - PRIMARY OUTPUT
   - Simple table: customer_id, client_id, bank_name
   - Each customer has 1 or 2 rows (cards)
   - If 2 cards, they are from different banks

2. final_clean_customers_summary.csv
   - One row per customer with aggregated information
   
3. customer_card_combinations_detailed.csv
   - Full details including spending, cheque count, etc.

CONSTRAINTS ENFORCED:
- Each customer has <=2 client_ids (bank cards) from <=2 banks
- If 2 cards, they must be from 2 different banks (BOG or TBC)
- Only cards with >=3 transactions OR >$35 spending
- Shared cards (used by multiple customers) are removed



In [4]:
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import pickle
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Configuration
DATA_DIR = Path("analysis_data")
CHECKPOINT_DIR = DATA_DIR / "checkpoints"
DATA_DIR.mkdir(exist_ok=True)
CHECKPOINT_DIR.mkdir(exist_ok=True)

class CheckpointManager:
    def __init__(self, process_name, parameters=None):
        self.process_name = process_name
        self.parameters = parameters or {}
        param_hash = hash(str(sorted(self.parameters.items())))
        self.checkpoint_file = CHECKPOINT_DIR / f"{process_name}_{param_hash}_progress.json"
        self.progress = self.load_progress()
    
    def load_progress(self):
        if self.checkpoint_file.exists():
            with open(self.checkpoint_file, 'r') as f:
                data = json.load(f)
                if data.get("parameters") == self.parameters:
                    return data
                else:
                    print(f"Parameters changed for {self.process_name}, starting fresh")
                    return self._create_new_progress()
        return self._create_new_progress()
    
    def _create_new_progress(self):
        return {
            "parameters": self.parameters,
            "completed_chunks": [], 
            "status": "not_started", 
            "last_chunk": -1
        }
    
    def save_progress(self, chunk_info):
        self.progress["completed_chunks"].append(chunk_info)
        self.progress["last_chunk"] = len(self.progress["completed_chunks"]) - 1
        self.progress["status"] = "in_progress"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def mark_completed(self):
        self.progress["status"] = "completed"
        self.progress["parameters"] = self.parameters
        with open(self.checkpoint_file, 'w') as f:
            json.dump(self.progress, f, indent=2)
    
    def is_chunk_completed(self, chunk_id):
        return any(chunk["chunk_id"] == chunk_id for chunk in self.progress["completed_chunks"])
    
    def get_completed_chunks(self):
        return [chunk["filename"] for chunk in self.progress["completed_chunks"]]

def save_dataframe(df, filename_base):
    """Save dataframe with best available format"""
    try:
        parquet_file = DATA_DIR / f"{filename_base}.parquet"
        df.to_parquet(parquet_file, index=False)
        return parquet_file
    except ImportError:
        pickle_file = DATA_DIR / f"{filename_base}.pkl"
        with open(pickle_file, 'wb') as f:
            pickle.dump(df, f)
        return pickle_file

def load_dataframe(filename_base):
    """Load dataframe from best available format"""
    for ext, loader in [('.parquet', pd.read_parquet), ('.pkl', lambda x: pickle.load(open(x, 'rb')))]:
        file_path = DATA_DIR / f"{filename_base}{ext}"
        if file_path.exists():
            return loader(file_path)
    return None

def connect_server(ip, database):
    conn_str = (
        "Driver={SQL Server};"
        "Server=" + ip + ";" + 
        "Database=" + database + ";" + 
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    return conn, cursor

def sql_to_pandas(conn, cursor, sql_string):
    cursor.execute(sql_string)
    rows = cursor.fetchall()
    if rows:
        return pd.DataFrame.from_records(rows, columns=[desc[0] for desc in cursor.description])
    return pd.DataFrame()

def close_conn(conn, cursor):
    cursor.close()
    conn.close()

def view_bank_data(num_rows=10):
    """View sample of your bank data"""
    print("LOADING BANK DATA...")
    bank_df = load_dataframe("bank_data_combined")
    
    if bank_df is None:
        print("ERROR: Could not find bank_data_combined.parquet or .pkl")
        return None
    
    print(f"\nBANK DATA OVERVIEW:")
    print(f"  Total rows: {len(bank_df):,}")
    print(f"  Columns: {list(bank_df.columns)}")
    print(f"\nColumn types:")
    for col, dtype in bank_df.dtypes.items():
        print(f"  {col}: {dtype}")
    
    print(f"\nFirst {num_rows} rows:")
    print(bank_df.head(num_rows))
    
    print(f"\nUnique values per column:")
    for col in bank_df.columns:
        unique_count = bank_df[col].nunique()
        print(f"  {col}: {unique_count:,} unique values")
    
    if 'bank_name' in bank_df.columns:
        print(f"\nBank distribution:")
        print(bank_df['bank_name'].value_counts())
    
    return bank_df

def view_facts_data(num_rows=10):
    """View sample of your facts data"""
    print("LOADING FACTS DATA...")
    facts_df = load_dataframe("facts_data_2025_combined_customer_id")
    
    if facts_df is None:
        print("ERROR: Could not find facts_data_2025_combined_customer_id.parquet or .pkl")
        return None
    
    print(f"\nFACTS DATA OVERVIEW:")
    print(f"  Total rows: {len(facts_df):,}")
    print(f"  Columns: {list(facts_df.columns)}")
    print(f"\nColumn types:")
    for col, dtype in facts_df.dtypes.items():
        print(f"  {col}: {dtype}")
    
    print(f"\nFirst {num_rows} rows:")
    print(facts_df.head(num_rows))
    
    print(f"\nKey metrics:")
    print(f"  Unique customers: {facts_df['customer_id'].nunique():,}")
    print(f"  Unique cheques: {facts_df['cheque_id'].nunique():,}")
    print(f"  Unique reference numbers: {facts_df['reference_number'].nunique():,}")
    
    return facts_df

def analyze_loyalty_bank_cards_corrected(facts_df, bank_df):
    """
    CORRECTED analysis using client as bank card identifier
    
    Key changes:
    - reference_number is ONLY used to link tables
    - client (or client_id) is the actual bank card number
    - All grouping and filtering is done by customer_id + bank_card_id
    """
    print("\n=== CORRECTED LOYALTY BANK CARD ANALYSIS ===")
    
    if facts_df is None or bank_df is None or facts_df.empty or bank_df.empty:
        print("Invalid input data")
        return None, None
    
    # DIAGNOSTIC: Check what columns we actually have
    print("\n--- DIAGNOSTIC: CHECKING AVAILABLE COLUMNS ---")
    print(f"Facts columns: {list(facts_df.columns)}")
    print(f"Bank columns: {list(bank_df.columns)}")
    print(f"\nBank data sample (first row):")
    print(bank_df.head(1).to_dict('records'))
    
    # Determine the correct column name for bank card identifier
    bank_card_column = None
    if 'client_id' in bank_df.columns:
        bank_card_column = 'client_id'
    elif 'client' in bank_df.columns:
        bank_card_column = 'client'
    else:
        print("\nERROR: Cannot find bank card identifier column!")
        print("Looking for 'client_id' or 'client' in bank_df columns")
        print(f"Available columns: {list(bank_df.columns)}")
        return None, None
    
    print(f"\nUsing '{bank_card_column}' as bank card identifier")
    
    # STEP 0: AGGREGATE FACTS DATA BY CHEQUE_ID
    print("\n--- STEP 0: AGGREGATING FACTS DATA BY CHEQUE ---")
    
    print(f"Raw transaction lines: {len(facts_df):,}")
    print(f"Unique cheques: {facts_df['cheque_id'].nunique():,}")
    print(f"Unique customers: {facts_df['customer_id'].nunique():,}")
    print(f"Unique reference numbers: {facts_df['reference_number'].nunique():,}")
    
    cheque_totals = facts_df.groupby(['cheque_id', 'customer_id', 'reference_number']).agg({
        'total_price': 'sum'
    }).reset_index()
    
    cheque_totals.columns = ['cheque_id', 'customer_id', 'reference_number', 'cheque_total']
    
    print(f"Cheque-level data: {len(cheque_totals):,} records")
    print(f"Average spending per cheque: ${cheque_totals['cheque_total'].mean():.2f}")
    
    # STEP 1: JOIN WITH BANK DATA TO GET BANK CARD ID
    print(f"\n--- STEP 1: JOINING WITH BANK DATA TO GET {bank_card_column.upper()} ---")
    
    cheque_totals['reference_number'] = cheque_totals['reference_number'].astype(str)
    bank_df['reference_number'] = bank_df['reference_number'].astype(str)
    
    # Select columns from bank_df
    bank_columns = ['reference_number', bank_card_column, 'bank_name']
    if 'client' in bank_df.columns and bank_card_column != 'client':
        bank_columns.append('client')
    
    merged_df = cheque_totals.merge(
        bank_df[bank_columns], 
        on='reference_number', 
        how='inner'
    )
    
    # Rename to standardize as 'bank_card_id'
    merged_df = merged_df.rename(columns={bank_card_column: 'bank_card_id'})
    
    print(f"After join:")
    print(f"  Successfully merged cheque records: {len(merged_df):,}")
    print(f"  Unique customers with bank data: {merged_df['customer_id'].nunique():,}")
    print(f"  Unique bank cards: {merged_df['bank_card_id'].nunique():,}")
    print(f"  Unique reference numbers used for linking: {merged_df['reference_number'].nunique():,}")
    print(f"  Available banks: {sorted(merged_df['bank_name'].unique())}")
    
    if len(merged_df) == 0:
        print("No matching data found!")
        return None, None
    
    # STEP 2: VALIDATE CUSTOMER-CARD COMBINATIONS
    print("\n--- STEP 2: VALIDATING CUSTOMER-CARD COMBINATIONS ---")
    print("Validation: customer_id must have >=3 cheques OR >$35 total spending per bank_card_id")
    
    agg_dict = {
        'cheque_id': 'nunique',
        'cheque_total': 'sum',
        'bank_name': 'first',
        'reference_number': lambda x: list(x.unique())
    }
    
    if 'client' in merged_df.columns:
        agg_dict['client'] = 'first'
    
    customer_card_usage = merged_df.groupby(['customer_id', 'bank_card_id']).agg(agg_dict).reset_index()
    
    col_names = ['customer_id', 'bank_card_id', 'cheque_count', 
                 'total_spending', 'bank_name', 'reference_numbers']
    if 'client' in merged_df.columns:
        col_names.append('client')
    
    customer_card_usage.columns = col_names
    
    print(f"Customer-card combinations: {len(customer_card_usage):,}")
    print(f"Average cheques per combination: {customer_card_usage['cheque_count'].mean():.1f}")
    print(f"Average spending per combination: ${customer_card_usage['total_spending'].mean():.2f}")
    
    valid_customer_cards = customer_card_usage[
        (customer_card_usage['cheque_count'] >= 3) | 
        (customer_card_usage['total_spending'] > 35)
    ].copy()
    
    invalid_customer_cards = customer_card_usage[
        (customer_card_usage['cheque_count'] < 3) & 
        (customer_card_usage['total_spending'] <= 35)
    ].copy()
    
    print(f"Valid customer-card combinations: {len(valid_customer_cards):,}")
    print(f"Invalid customer-card combinations: {len(invalid_customer_cards):,}")
    print(f"Validation rate: {len(valid_customer_cards)/len(customer_card_usage)*100:.1f}%")
    
    # STEP 3: FILTER TO BOG AND TBC BANKS ONLY
    print("\n--- STEP 3: FILTERING TO BOG AND TBC BANKS ---")
    
    target_banks = ['BOG', 'TBC']
    bog_tbc_valid_cards = valid_customer_cards[
        valid_customer_cards['bank_name'].isin(target_banks)
    ].copy()
    
    print(f"Valid combinations with BOG/TBC banks: {len(bog_tbc_valid_cards):,}")
    print("Bank distribution:")
    bank_dist = bog_tbc_valid_cards['bank_name'].value_counts()
    for bank, count in bank_dist.items():
        print(f"  {bank}: {count:,} combinations")
    
    if len(bog_tbc_valid_cards) == 0:
        print("No valid BOG/TBC combinations found!")
        return None, None
    
    # STEP 4: FILTER CUSTOMERS WITH <=2 VALID CARDS FROM <=2 BANKS
    print("\n--- STEP 4: FILTERING CUSTOMERS BY CARD AND BANK LIMITS ---")
    print("Requirements:")
    print("  - <=2 different valid bank_card_ids from <=2 different banks")
    print("  - If 2 cards, they MUST be from 2 different banks")
    
    customer_summary = bog_tbc_valid_cards.groupby('customer_id').agg({
        'bank_card_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': ['nunique', lambda x: list(x.unique())],
        'cheque_count': 'sum',
        'total_spending': 'sum'
    }).reset_index()
    
    customer_summary.columns = ['customer_id', 'valid_cards_count', 'valid_cards_list',
                                'different_banks_count', 'different_banks_list', 
                                'total_cheques', 'total_spending']
    
    print(f"Customers with valid BOG/TBC cards: {len(customer_summary):,}")
    
    eligible_customers = customer_summary[
        (customer_summary['valid_cards_count'] <= 2) & 
        (customer_summary['different_banks_count'] <= 2) &
        (
            (customer_summary['valid_cards_count'] == 1) |
            ((customer_summary['valid_cards_count'] == 2) & (customer_summary['different_banks_count'] == 2))
        )
    ].copy()
    
    print(f"After filtering: {len(eligible_customers):,} customers meeting requirements")
    
    if len(eligible_customers) == 0:
        print("No customers meet the criteria!")
        return None, None
    
    print("\nValid cards per customer distribution:")
    cards_after = eligible_customers['valid_cards_count'].value_counts().sort_index()
    for cards, count in cards_after.items():
        print(f"  {count:,} customers have {cards} valid card(s)")
    
    # STEP 5: REMOVE SHARED BANK_CARD_IDS
    print("\n--- STEP 5: REMOVING SHARED BANK_CARD_IDS ---")
    print("Remove bank_card_ids used by multiple customers")
    
    eligible_customer_ids = set(eligible_customers['customer_id'])
    eligible_customer_cards = bog_tbc_valid_cards[
        bog_tbc_valid_cards['customer_id'].isin(eligible_customer_ids)
    ].copy()
    
    card_usage_analysis = eligible_customer_cards.groupby('bank_card_id').agg({
        'customer_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': 'first',
        'cheque_count': 'sum',
        'total_spending': 'sum'
    }).reset_index()
    
    card_usage_analysis.columns = ['bank_card_id', 'customer_count', 'customer_list',
                                   'bank_name', 'total_cheques', 'total_spending']
    
    shared_cards = card_usage_analysis[
        card_usage_analysis['customer_count'] > 1
    ]['bank_card_id'].tolist()
    
    unique_cards = card_usage_analysis[
        card_usage_analysis['customer_count'] == 1
    ]['bank_card_id'].tolist()
    
    print(f"Cards used by single customer: {len(unique_cards):,}")
    print(f"Cards shared by multiple customers: {len(shared_cards):,}")
    
    clean_customer_cards = eligible_customer_cards[
        ~eligible_customer_cards['bank_card_id'].isin(shared_cards)
    ].copy()
    
    print(f"Customer-card combinations after removing shared cards: {len(clean_customer_cards):,}")
    
    # STEP 6: CREATE FINAL CLEAN DATASET
    print("\n--- STEP 6: CREATING FINAL CLEAN DATASET ---")
    
    final_agg_dict = {
        'bank_card_id': ['nunique', lambda x: list(x.unique())],
        'bank_name': ['nunique', lambda x: list(x.unique())],
        'cheque_count': 'sum',
        'total_spending': 'sum',
        'reference_numbers': lambda x: list(set([ref for refs in x for ref in refs]))
    }
    
    if 'client' in clean_customer_cards.columns:
        final_agg_dict['client'] = lambda x: list(x.unique())
    
    final_customer_summary = clean_customer_cards.groupby('customer_id').agg(final_agg_dict).reset_index()
    
    col_names = ['customer_id', 'final_cards_count', 'final_cards_list',
                'different_banks_count', 'different_banks_list', 
                'total_cheques', 'total_spending', 'reference_numbers']
    
    if 'client' in clean_customer_cards.columns:
        col_names.append('clients_list')
    
    final_customer_summary.columns = col_names
    
    final_customer_summary = final_customer_summary[
        final_customer_summary['final_cards_count'] > 0
    ].copy()
    
    print(f"Final clean customers: {len(final_customer_summary):,}")
    
    constraint_violations = final_customer_summary[
        (final_customer_summary['final_cards_count'] == 2) & 
        (final_customer_summary['different_banks_count'] != 2)
    ]
    
    if len(constraint_violations) > 0:
        print(f"WARNING: {len(constraint_violations)} customers have 2 cards from same bank")
        final_customer_summary = final_customer_summary[
            ~final_customer_summary['customer_id'].isin(constraint_violations['customer_id'])
        ].copy()
        print(f"Removed violations, final customers: {len(final_customer_summary):,}")
    
    print("\nFINAL DISTRIBUTION - Cards per customer:")
    final_cards_dist = final_customer_summary['final_cards_count'].value_counts().sort_index()
    for cards, count in final_cards_dist.items():
        print(f"  {count:,} customers have {cards} card(s)")
    
    # Final verification
    max_cards = final_customer_summary['final_cards_count'].max()
    max_banks = final_customer_summary['different_banks_count'].max()
    two_cards_same_bank = len(final_customer_summary[
        (final_customer_summary['final_cards_count'] == 2) & 
        (final_customer_summary['different_banks_count'] == 1)
    ])
    
    final_customer_ids = set(final_customer_summary['customer_id'])
    final_clean_cards = clean_customer_cards[
        clean_customer_cards['customer_id'].isin(final_customer_ids)
    ].copy()
    
    final_card_sharing = final_clean_cards.groupby('bank_card_id')['customer_id'].nunique()
    shared_in_final = (final_card_sharing > 1).sum()
    
    print(f"\nFINAL VERIFICATION:")
    print(f"  Maximum cards per customer: {max_cards} (requirement: <=2) {'PASS' if max_cards <= 2 else 'FAIL'}")
    print(f"  Maximum banks per customer: {max_banks} (requirement: <=2) {'PASS' if max_banks <= 2 else 'FAIL'}")
    print(f"  Customers with 2 cards from same bank: {two_cards_same_bank} (requirement: 0) {'PASS' if two_cards_same_bank == 0 else 'FAIL'}")
    print(f"  Shared cards in final dataset: {shared_in_final} (requirement: 0) {'PASS' if shared_in_final == 0 else 'FAIL'}")
    
    all_passed = max_cards <= 2 and max_banks <= 2 and two_cards_same_bank == 0 and shared_in_final == 0
    print(f"  OVERALL: {'ALL REQUIREMENTS SATISFIED' if all_passed else 'REQUIREMENTS NOT MET'}")
    
    # CREATE CUSTOMER-CARD PAIRS OUTPUT
    print("\n--- CREATING CUSTOMER-CARD PAIRS ---")
    
    customer_card_pairs = final_clean_cards[['customer_id', 'bank_card_id', 'bank_name']].drop_duplicates()
    print(f"Total customer-card pairs: {len(customer_card_pairs):,}")
    
    pairs_per_customer = customer_card_pairs.groupby('customer_id').size()
    print(f"Pairs per customer: min={pairs_per_customer.min()}, max={pairs_per_customer.max()}")
    
    results = {
        'raw_transaction_lines': len(facts_df),
        'total_cheques': len(cheque_totals),
        'total_loyalty_customers': cheque_totals['customer_id'].nunique(),
        'customers_with_bank_data': merged_df['customer_id'].nunique(),
        'total_customer_card_combinations': len(customer_card_usage),
        'valid_customer_card_combinations': len(valid_customer_cards),
        'bog_tbc_valid_combinations': len(bog_tbc_valid_cards),
        'eligible_customers_before_shared_removal': len(eligible_customers),
        'shared_cards_count': len(shared_cards),
        'final_clean_customers': len(final_customer_summary),
        'final_clean_combinations': len(final_clean_cards),
        'bank_card_column_used': bank_card_column,
        
        'cheque_totals': cheque_totals,
        'customer_card_usage': customer_card_usage,
        'valid_customer_cards': valid_customer_cards,
        'invalid_customer_cards': invalid_customer_cards,
        'bog_tbc_valid_cards': bog_tbc_valid_cards,
        'eligible_customers': eligible_customers,
        'card_usage_analysis': card_usage_analysis,
        'shared_cards_info': card_usage_analysis[card_usage_analysis['customer_count'] > 1],
        'shared_cards_list': shared_cards,
        'final_clean_cards': final_clean_cards,
        'final_customer_summary': final_customer_summary,
        'customer_card_pairs': customer_card_pairs,
        'merged_data': merged_df,
        'final_cards_distribution': final_cards_dist
    }
    
    return results, final_customer_summary

def save_corrected_results(results, output_dir="./data"):
    """Save all results to CSV files"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    saved_files = []
    
    try:
        bank_card_col = results.get('bank_card_column_used', 'client')
        print(f"\nNote: Bank card column in original data was '{bank_card_col}'")
        print(f"Outputs will show this as 'bank_card_id'\n")
        
        # PRIMARY OUTPUT: Customer-Card Pairs
        if 'customer_card_pairs' in results and not results['customer_card_pairs'].empty:
            file_path = output_path / "customer_card_pairs.csv"
            results['customer_card_pairs'].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
            print(f"PRIMARY OUTPUT SAVED: {file_path}")
            print(f"  Rows: {len(results['customer_card_pairs']):,}")
            print(f"  Columns: {list(results['customer_card_pairs'].columns)}")
        
        # Customer summary
        if 'final_customer_summary' in results and not results['final_customer_summary'].empty:
            summary_csv = results['final_customer_summary'].copy()
            
            summary_csv['final_cards_list_str'] = summary_csv['final_cards_list'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            summary_csv['different_banks_list_str'] = summary_csv['different_banks_list'].apply(
                lambda x: ', '.join(x) if isinstance(x, list) else str(x))
            
            if 'clients_list' in summary_csv.columns:
                summary_csv['clients_list_str'] = summary_csv['clients_list'].apply(
                    lambda x: ', '.join(x) if isinstance(x, list) else str(x))
            
            summary_csv['reference_numbers_str'] = summary_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x[:20])) + (f' (+{len(x)-20} more)' if len(x) > 20 else '') if isinstance(x, list) else str(x))
            
            csv_columns = ['customer_id', 'final_cards_count', 'different_banks_count', 'total_cheques', 
                          'total_spending', 'final_cards_list_str', 'different_banks_list_str', 'reference_numbers_str']
            
            if 'clients_list_str' in summary_csv.columns:
                csv_columns.insert(-1, 'clients_list_str')
            
            file_path = output_path / "final_clean_customers_summary.csv"
            summary_csv[csv_columns].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Detailed combinations
        if 'final_clean_cards' in results and not results['final_clean_cards'].empty:
            clean_cards_csv = results['final_clean_cards'].copy()
            clean_cards_csv['reference_numbers_str'] = clean_cards_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            output_cols = ['customer_id', 'bank_card_id', 'bank_name', 'cheque_count', 
                          'total_spending', 'reference_numbers_str']
            
            if 'client' in clean_cards_csv.columns:
                output_cols.insert(3, 'client')
            
            file_path = output_path / "customer_card_combinations_detailed.csv"
            clean_cards_csv[output_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Shared cards
        if 'shared_cards_info' in results and not results['shared_cards_info'].empty:
            file_path = output_path / "shared_cards_removed.csv"
            shared_csv = results['shared_cards_info'].copy()
            shared_csv['customer_list_str'] = shared_csv['customer_list'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            csv_cols = ['bank_card_id', 'customer_count', 'bank_name', 'total_cheques', 
                       'total_spending', 'customer_list_str']
            shared_csv[csv_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Invalid combinations
        if 'invalid_customer_cards' in results and not results['invalid_customer_cards'].empty:
            file_path = output_path / "invalid_customer_card_combinations.csv"
            invalid_csv = results['invalid_customer_cards'].copy()
            invalid_csv['reference_numbers_str'] = invalid_csv['reference_numbers'].apply(
                lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))
            
            output_cols = ['customer_id', 'bank_card_id', 'cheque_count', 'total_spending', 
                          'bank_name', 'reference_numbers_str']
            invalid_csv[output_cols].to_csv(file_path, index=False)
            saved_files.append(str(file_path))
        
        # Summary
        analysis_summary = {
            'total_loyalty_customers': results['total_loyalty_customers'],
            'final_clean_customers': results['final_clean_customers'],
            'success_rate_percent': results['final_clean_customers']/results['total_loyalty_customers']*100,
            'shared_cards_removed': results['shared_cards_count'],
            'validation_rate_percent': results['valid_customer_card_combinations']/results['total_customer_card_combinations']*100,
            'bank_card_column_in_original_data': results.get('bank_card_column_used', 'client')
        }
        
        file_path = output_path / "analysis_summary.csv"
        pd.DataFrame([analysis_summary]).to_csv(file_path, index=False)
        saved_files.append(str(file_path))
        
        print(f"\nSAVED {len(saved_files)} FILES:")
        for file_path in saved_files:
            print(f"   {file_path}")
                
    except Exception as e:
        print(f"Error saving files: {e}")
        import traceback
        traceback.print_exc()
        return False
    
    return True

def run_full_analysis():
    """Run complete analysis using PRESAVED data files"""
    print("RUNNING COMPLETE ANALYSIS - WITH CLIENT AS BANK CARD ID")
    print("="*60)
    print("Looking for presaved data files...")
    
    facts_df = load_dataframe("facts_data_2025_combined_customer_id")
    bank_df = load_dataframe("bank_data_combined")
    
    if facts_df is None or bank_df is None:
        print("\nERROR: No presaved data found!")
        print("Expected files in './analysis_data/' directory:")
        print("  - facts_data_2025_combined_customer_id.parquet (or .pkl)")
        print("  - bank_data_combined.parquet (or .pkl)")
        return None
    
    print("\nSUCCESS: Found presaved data!")
    print(f"  Facts data: {len(facts_df):,} records")
    print(f"  Bank data: {len(bank_df):,} records")
    
    results, summary = analyze_loyalty_bank_cards_corrected(facts_df, bank_df)
    
    if results is None:
        print("Analysis failed")
        return None
    
    save_corrected_results(results)
    
    return results, summary

def run_analysis_with_dataframes(facts_df, bank_df):
    """Run analysis using dataframes you already have loaded"""
    print("RUNNING ANALYSIS WITH PROVIDED DATAFRAMES")
    print("="*60)
    
    if facts_df is None or bank_df is None:
        print("ERROR: One or both dataframes are None")
        return None
    
    print(f"Facts data: {len(facts_df):,} records")
    print(f"Bank data: {len(bank_df):,} records")
    
    results, summary = analyze_loyalty_bank_cards_corrected(facts_df, bank_df)
    
    if results is None:
        print("Analysis failed")
        return None
    
    save_corrected_results(results)
    
    return results, summary

def show_usage():
    print("""
USAGE - CORRECTED VERSION WITH CLIENT AS BANK CARD NUMBER

==================================================================
VIEW YOUR DATA FIRST (RECOMMENDED)
==================================================================
>>> bank_df = view_bank_data()
>>> facts_df = view_facts_data()

==================================================================
RUN ANALYSIS
==================================================================
Option 1 - Use presaved data:
>>> results, summary = run_full_analysis()

Option 2 - Use dataframes in memory:
>>> results, summary = run_analysis_with_dataframes(facts_df, bank_df)

==================================================================
OUTPUT FILES
==================================================================
Primary: customer_card_pairs.csv (customer_id, bank_card_id, bank_name)
""")

if __name__ == "__main__":
    show_usage()


USAGE - CORRECTED VERSION WITH CLIENT AS BANK CARD NUMBER

VIEW YOUR DATA FIRST (RECOMMENDED)
>>> bank_df = view_bank_data()
>>> facts_df = view_facts_data()

RUN ANALYSIS
Option 1 - Use presaved data:
>>> results, summary = run_full_analysis()

Option 2 - Use dataframes in memory:
>>> results, summary = run_analysis_with_dataframes(facts_df, bank_df)

OUTPUT FILES
Primary: customer_card_pairs.csv (customer_id, bank_card_id, bank_name)



In [5]:
results, summary = run_full_analysis()

RUNNING COMPLETE ANALYSIS - WITH CLIENT AS BANK CARD ID
Looking for presaved data files...

SUCCESS: Found presaved data!
  Facts data: 24,519,920 records
  Bank data: 35,362,470 records

=== CORRECTED LOYALTY BANK CARD ANALYSIS ===

--- DIAGNOSTIC: CHECKING AVAILABLE COLUMNS ---
Facts columns: ['cheque_id', 'is_loyalty', 'discount_card_no', 'customer_id', 'reference_number', 'total_price', 'item_count']
Bank columns: ['reference_number', 'transaction_amount', 'bank_name', 'client']

Bank data sample (first row):
[{'reference_number': '522310265367', 'transaction_amount': Decimal('10.98'), 'bank_name': 'BOG', 'client': '5e2df56dbb9a6573cff6180d6c0a21d05fb1ca2e9c9121a7f5275e6afd853415'}]

Using 'client' as bank card identifier

--- STEP 0: AGGREGATING FACTS DATA BY CHEQUE ---
Raw transaction lines: 24,519,920
Unique cheques: 24,519,797
Unique customers: 691,124
Unique reference numbers: 24,477,555
Cheque-level data: 24,519,920 records
Average spending per cheque: $17.82

--- STEP 1: JOI

In [6]:
 #show_status()

In [7]:
#quick_test()

In [8]:
# load_all_data_safely()

In [9]:
results, summary = run_complete_analysis_with_validation()

NameError: name 'run_complete_analysis_with_validation' is not defined

In [ ]:
import pandas as pd
from pathlib import Path

# Load the facts data
DATA_DIR = Path("analysis_data")
facts_df = pd.read_parquet(DATA_DIR / "facts_data_2025_combined_customer_id.parquet")

# Check the is_loyalty column
print("FACTS DATA - IS_LOYALTY BREAKDOWN")
print("="*60)
print(f"Total rows: {len(facts_df):,}")

if 'is_loyalty' in facts_df.columns:
    print(f"\nis_loyalty breakdown:")
    print(facts_df['is_loyalty'].value_counts())
    print(f"\nPercentages:")
    print(facts_df['is_loyalty'].value_counts(normalize=True) * 100)
    
    # Check customer_id for non-loyalty transactions
    print(f"\n--- NON-LOYALTY TRANSACTIONS (is_loyalty = False) ---")
    non_loyalty = facts_df[facts_df['is_loyalty'] == False]
    print(f"Total non-loyalty rows: {len(non_loyalty):,}")
    print(f"Non-loyalty with customer_id (should be 0): {non_loyalty['customer_id'].notna().sum():,}")
    print(f"Non-loyalty without customer_id (expected): {non_loyalty['customer_id'].isna().sum():,}")
    
    print(f"\n--- LOYALTY TRANSACTIONS (is_loyalty = True) ---")
    loyalty = facts_df[facts_df['is_loyalty'] == True]
    print(f"Total loyalty rows: {len(loyalty):,}")
    print(f"Loyalty with customer_id: {loyalty['customer_id'].notna().sum():,}")
    print(f"Unique loyalty customers: {loyalty['customer_id'].nunique():,}")
    
    # Show sample of each
    print(f"\n--- SAMPLE NON-LOYALTY TRANSACTION ---")
    print(non_loyalty.head(1))
    
    print(f"\n--- SAMPLE LOYALTY TRANSACTION ---")
    print(loyalty.head(1))
    
else:
    print("ERROR: 'is_loyalty' column not found!")
    print(f"Available columns: {list(facts_df.columns)}")

FACTS DATA - IS_LOYALTY BREAKDOWN
Total rows: 24,519,920

is_loyalty breakdown:
is_loyalty
1    24519920
Name: count, dtype: int64

Percentages:
is_loyalty
1    100.0
Name: proportion, dtype: float64

--- NON-LOYALTY TRANSACTIONS (is_loyalty = False) ---
Total non-loyalty rows: 0
Non-loyalty with customer_id (should be 0): 0
Non-loyalty without customer_id (expected): 0

--- LOYALTY TRANSACTIONS (is_loyalty = True) ---
Total loyalty rows: 24,519,920
Loyalty with customer_id: 24,519,920
Unique loyalty customers: 691,124

--- SAMPLE NON-LOYALTY TRANSACTION ---
Empty DataFrame
Columns: [cheque_id, is_loyalty, discount_card_no, customer_id, reference_number, total_price, item_count]
Index: []

--- SAMPLE LOYALTY TRANSACTION ---
   cheque_id  is_loyalty discount_card_no               customer_id  \
0  622115623           1     888800035608  6084819b2c5031000fbcf349   

  reference_number total_price  item_count  
0     500407689483       16.95           5  
